In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms 

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)  # Fixed closing parenthesis

test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)  # Fixed typo in 'download'

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)


100%|████████████████████████████████████████| 170M/170M [42:47<00:00, 66.4kB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
image, label = train_data[0]

In [5]:
image.size()

torch.Size([3, 32, 32])

In [6]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [7]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 


In [8]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
for epoch in range(30):
    print(f'Training epoch {epoch}... ')
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0... 
Loss: 2.1948
Training epoch 1... 
Loss: 1.7831
Training epoch 2... 
Loss: 1.5426
Training epoch 3... 
Loss: 1.4156
Training epoch 4... 
Loss: 1.3257
Training epoch 5... 
Loss: 1.2459
Training epoch 6... 
Loss: 1.1771
Training epoch 7... 
Loss: 1.1128
Training epoch 8... 
Loss: 1.0575
Training epoch 9... 
Loss: 1.0062
Training epoch 10... 
Loss: 0.9645
Training epoch 11... 
Loss: 0.9242
Training epoch 12... 
Loss: 0.8860
Training epoch 13... 
Loss: 0.8519
Training epoch 14... 
Loss: 0.8242
Training epoch 15... 
Loss: 0.7904
Training epoch 16... 
Loss: 0.7602
Training epoch 17... 
Loss: 0.7382
Training epoch 18... 
Loss: 0.7100
Training epoch 19... 
Loss: 0.6878
Training epoch 20... 
Loss: 0.6616
Training epoch 21... 
Loss: 0.6420
Training epoch 22... 
Loss: 0.6193
Training epoch 23... 
Loss: 0.5966
Training epoch 24... 
Loss: 0.5767
Training epoch 25... 
Loss: 0.5571
Training epoch 26... 
Loss: 0.5325
Training epoch 27... 
Loss: 0.5144
Training epoch 28... 
Loss: 0.

In [13]:
torch.save(net.state_dict(), 'trained_net.pth')

net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth', weights_only=True))


<All keys matched successfully>

In [15]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct/total
print(f'Accuracy: {accuracy}%')


Accuracy: 68.06%


In [25]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_images(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image 
image_paths = ['example1.jpg', 'example2.jpg']
images = [load_images(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}')

Prediction: dog
Prediction: plane
